In [ ]:
from aws_tools import *

# Update state machine and job def

create_aws_resources()

In [ ]:
# Get the s3 addresses for the dataset

prefix = 'south_atlantic'
resolution = '5min'
staging_bucket = 'kerchunk-staging'
run_name = 'south_atlantic-5min-2'

files = get_dataset('nrel-pds-wtk', prefix=prefix, resolution=resolution)
files

In [ ]:
# Generate the state machine input for this dataset

s3_comb_ref_file = f'wtk/{prefix}/kerchunk_{resolution}_ref_s3.json'
az_comb_ref_file = f'wtk/{prefix}/kerchunk_{resolution}_ref.json'
create_state_machine_input(files, staging_bucket, s3_comb_ref_file, az_comb_ref_file, run_name=run_name)

In [ ]:
# Run the state machine

run_state_machine('kerchunk-h5', run_name=run_name)

If the state machine fully executed without error, then there should now be a set of transformed h5 files, s3 refs and az refs, as well as a combined s3 ref file in the staging bucket. Use the test_staging.ipynb notebook to verify that the transformation was successful by loading the combined s3 ref file.

Once you are satisfied, continue to the next cell to copy the data to Azure and generate the combined az ref file.

Make sure to update the .env file with AWS credentials!

In [ ]:
copy_s3_dataset_to_azure(files, staging_bucket, dry_run=False)

In [ ]:
comb_ref_file = f'wtk/{prefix}/kerchunk_{resolution}_ref.json'
create_combined_ref(files, staging_bucket, comb_ref_file=comb_ref_file, remote_protocol='abfs')

Once these tasks have finished, you can open the wtk example notebook and verify that the dataset can now be loaded from Azure.